# Exploring the relationg between the tile and ROC space

> TODO: explications

In [13]:
%matplotlib tk

import os
import json
import numpy as np
from utils import get_colors
import matplotlib.pyplot as plt
from matplotlib.backend_bases import MouseButton

from sorbetto.core.entity import Entity
from sorbetto.ranking.ranking_score import RankingScore
from sorbetto.tile.empty_tile import EmptyTile
from sorbetto.core.importance import Importance
from sorbetto.performance.two_class_classification_performance import TwoClassClassificationPerformance
from sorbetto.parameterization.parameterization_default import ParameterizationDefault

In [14]:
# loading data

PATH_DATA = os.path.join(os.getcwd(), 'data', 'semantic_segmentation_methods.json')

MAP_DATASETS = {
    'Cityscapes': '♠',
    'ADE20K': '♥',
    'Pascal VOC 2012': '♦',
    'COCO-Stuff 164k': '♣'
}

def read_json(filePath:str):
    """
    Function to read json 
    """
    
    with open(filePath, 'r') as f:
        data = json.load(f)
    return data


def read_data(filePath:str = PATH_DATA):
    data = read_json(filePath)

    processed_data = []

    colors = get_colors(len(data))

    # convert each instance into an Entity
    for i, elem in enumerate(data):
        performance = TwoClassClassificationPerformance(
            ptp=elem['proba_tp'],
            ptn=elem['proba_tn'],
            pfp=elem['proba_fp'],
            pfn=elem['proba_fn']
        )

        entity = Entity(
            performance= performance,
            name = f"{elem['model']} {MAP_DATASETS[elem['learning_set']]}",
            color= colors[i]
        )

        processed_data.append(entity)

    return processed_data


data = read_data()

In [15]:
# selecting the entity that will be used to compute the tile

id_entity = 0

perf = data[id_entity].performance

In [16]:
# global variables

parameterization = ParameterizationDefault()
tile = EmptyTile("Tile", parameterization)
tile2 = EmptyTile("ROC", parameterization)
PRIOR_POS = 0.5

In [17]:
# helper functions

def showRankingScore(score, fig, ax, priorPos = PRIOR_POS):
        score.drawInROC(fig, ax, priorPos=priorPos, show_colorbar = False)

def get_ranking_score(a, b):
    itp = a; itn = 1-a; ifp = 1-b; ifn = b
    importance = Importance(itn=itn, ifp=ifp, ifn=ifn, itp=itp)
    return RankingScore(importance=importance)

def plot_roc(fig, ax, coords):
    importance = parameterization.getCanonicalImportance(*coords)
    rankScore = RankingScore(importance=importance)
    showRankingScore(rankScore, fig, ax)
    ax.set_box_aspect(1)
    ax.set_anchor('C')
    ax.set_title('ROC', fontsize=12, pad=20)

In [18]:
fig, ax = plt.subplots(1, 2, layout="constrained")
tile.draw(fig, ax[0])
tile2.draw(fig, ax[1])
right_spec = ax[1].get_subplotspec()


def on_move(event):
    if event.inaxes is not ax[0] or event.xdata is None or event.ydata is None:
        return

    fig.delaxes(ax[1])
    ax[1] = fig.add_subplot(right_spec)

    plot_roc(fig, ax[1], (event.xdata, event.ydata))

    fig.canvas.draw_idle()


cid_move = fig.canvas.mpl_connect('motion_notify_event', on_move)

plt.show()